# Playing around with Enformer and enformer_loader

## We can run VSCode in the farm!

In [2]:
import os

cwd = os.getcwd()
print(cwd)

/lustre/scratch126/gengen/teams/parts/sb79/enformer_loader_sgb


In [3]:
import socket

node_name = socket.gethostname()
print(node_name)

node-13-17


In [7]:
import multiprocess
cores = multiprocess.cpu_count()
print("Cores Available: " + str(cores))

Cores Available: 256


In [ ]:
# Test R code to check connection to a node
print("Hello, R is running successfully!")

# Check the R version
R.version.string

# Check the current working directory
getwd()
# Generate a simple plot
x <- rnorm(100)
y <- rnorm(100)
plot(x, y, main="Scatter plot of random points", xlab="X-axis", ylab="Y-axis")
quartz()  # Opens a new window for the plot (on Linux or macOS)
plot(1:10, 1:10)

R code is not that well suited to VSCode... but should be able to write scripts here anyway :)

## Let's inspect the enformer_loader output

In [1]:
# Import necessary packages
import numpy as np
import sys
import os
import pandas as pd
from Bio import SeqIO
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
import math
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import re
import csv

/software/hgi/installs/conda-audited/miniforge/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
#sequence import
import pysam
import pyfaidx

# import fasta file and create function for retrieving sequences
fa_loc = '/lustre/scratch126/gengen/projects/graft/Dataset/reference/hg38_galGal6_full/fasta/GRCh38.GRCg6a.full.renamed.merged.fa'
pyfaidx.Faidx(fa_loc)
fasta_open = pysam.Fastafile(fa_loc)

In [36]:
chromlength = []
chromname = []
with pysam.Fastafile(fa_loc) as fasta:
    for name in fasta.references:
        chromname.append(name)
        length = fasta.get_reference_length(name)
        chromlength.append(length)
        print(f"Chromosome {name} has length {length}")

Chromosome hg38_1 has length 248956422
Chromosome hg38_10 has length 133797422
Chromosome hg38_11 has length 135086622
Chromosome hg38_12 has length 133275309
Chromosome hg38_13 has length 114364328
Chromosome hg38_14 has length 107043718
Chromosome hg38_15 has length 101991189
Chromosome hg38_16 has length 90338345
Chromosome hg38_17 has length 83257441
Chromosome hg38_18 has length 80373285
Chromosome hg38_19 has length 58617616
Chromosome hg38_2 has length 242193529
Chromosome hg38_20 has length 64444167
Chromosome hg38_21 has length 46709983
Chromosome hg38_22 has length 50818468
Chromosome hg38_3 has length 198295559
Chromosome hg38_4 has length 190214555
Chromosome hg38_5 has length 181538259
Chromosome hg38_6 has length 170805979
Chromosome hg38_7 has length 159345973
Chromosome hg38_8 has length 145138636
Chromosome hg38_9 has length 138394717
Chromosome hg38_MT has length 16569
Chromosome hg38_X has length 156040895
Chromosome hg38_Y has length 57227415
Chromosome hg38_KI27072

In [37]:
# write to chromosome lengths file
output_file = "all_chrom_len.txt"
# Open the file in write mode
with open(output_file, "w") as file:
    # Zip the two vectors together and write them to the file
    for n, l in zip(chromname, chromlength):
        file.write(f"{n}\t{l}\n")



In [38]:
# write to chromosome lengths file
output_file = "all_chrom_names.txt"
# Open the file in write mode
with open(output_file, "w") as file:
    # Zip the two vectors together and write them to the file
    for n, l in zip(chromname, chromlength):
        file.write(f"{n}\n")



In [18]:
# let's import the .bed file from enformer_loader
path = 'tests/data/'
bed_df = pd.read_csv(path + 'test_dataset.bed', sep='\t', header=None)
bed_df

,0,1,2,3
0,chr1,880,920,"[1.0, 1.0]"
1,chr1,493,533,"[1.0, 1.0]"
2,chr2,390,430,"[2.200000047683716, 3.200000047683716]"
3,chr1,92,132,"[1.0, 1.0]"
4,chr1,458,498,"[1.0, 1.0]"
...,...,...,...,...
95,chr3,563,603,"[2.0, 2.0]"
96,chr1,319,359,"[1.0, 1.0]"
97,chr2,477,517,"[3.200000047683716, 3.200000047683716]"
98,chr2,21,61,"[0.800000011920929, 0.800000011920929]"


In [21]:
# see how we could generate a train/val/test dataset for enformer
gtf_file = "/lustre/scratch126/gengen/projects/graft/Dataset/reference/hg38_galGal6_chr1/gtf/GRCh38.GRCg6a.chr1.110.gtf"
gtf_df = pd.read_csv(gtf_file, sep="\t", comment="#", header=None)
# Give appropriate column names
gtf_df.columns = ['seqname', 'source', 'feature', 'start', 'end', '.', 'strand', '.', 'attributes']
gtf_df['seqname'].unique()

array(['hg38_1', 'hg38_10', 'hg38_11', 'hg38_12', 'hg38_13', 'hg38_14',
       'hg38_15', 'hg38_16', 'hg38_17', 'hg38_18', 'hg38_19', 'hg38_2',
       'hg38_20', 'hg38_21', 'hg38_22', 'hg38_3', 'hg38_4', 'hg38_5',
       'hg38_6', 'hg38_7', 'hg38_8', 'hg38_9', 'hg38_GL000009.2',
       'hg38_GL000194.1', 'hg38_GL000195.1', 'hg38_GL000205.2',
       'hg38_GL000213.1', 'hg38_GL000216.2', 'hg38_GL000218.1',
       'hg38_GL000219.1', 'hg38_GL000220.1', 'hg38_GL000225.1',
       'hg38_KI270442.1', 'hg38_KI270711.1', 'hg38_KI270713.1',
       'hg38_KI270721.1', 'hg38_KI270726.1', 'hg38_KI270727.1',
       'hg38_KI270728.1', 'hg38_KI270731.1', 'hg38_KI270733.1',
       'hg38_KI270734.1', 'hg38_KI270744.1', 'hg38_KI270750.1', 'hg38_MT',
       'hg38_X', 'hg38_Y', 'gg6_1'], dtype=object)

In [22]:
gtf_df.iloc[0:10,:]

,seqname,source,feature,start,end,.,strand,.,attributes
0,hg38_1,havana,gene,11869,14409,.,+,.,"gene_id ""ENSG00000290825""; gene_version ""1""; g..."
1,hg38_1,havana,transcript,11869,14409,.,+,.,"gene_id ""ENSG00000290825""; gene_version ""1""; t..."
2,hg38_1,havana,exon,11869,12227,.,+,.,"gene_id ""ENSG00000290825""; gene_version ""1""; t..."
3,hg38_1,havana,exon,12010,12057,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""6""; t..."
4,hg38_1,havana,transcript,12010,13670,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""6""; t..."
5,hg38_1,havana,gene,12010,13670,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""6""; g..."
6,hg38_1,havana,exon,12179,12227,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""6""; t..."
7,hg38_1,havana,exon,12613,12721,.,+,.,"gene_id ""ENSG00000290825""; gene_version ""1""; t..."
8,hg38_1,havana,exon,12613,12697,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""6""; t..."
9,hg38_1,havana,exon,12975,13052,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""6""; t..."


In [25]:
gtf_df.shape

(3629944, 9)

# Now let's try to create dataset for H3K4me3, with hg38_1 as train and hg38_2 as validation

We have test_1_valchr.txt containing the name of the validation chromosome (hg38_2)


- chrom_sizes: all_chrom_len.txt
- test.bw: /lustre/scratch126/gengen/teams/parts/sb79/final/bw/merged_H3K4me3_5c3_bamcov_50.bw
- out: test_train_dataset.bed / test_val_dataset.bed
- val_chroms.txt: test_1_valchr.txt
- exclude_chroms: test_exclude_chrom.txt (no hg38_1, hg38_2 BUT INCLUDE gg6_1 as we need it for test)
- options as default - 40 train sequences, 10 val sequences

we get test1_train_dataset.bed and test1_test_dataset.bed

In [3]:
path = 'tests/data/'
bed_df = pd.read_csv(path + 'test1_train_dataset.bed', sep='\t', header=None)
bed_df

,0,1,2,3
0,hg38_1,144671580,144786268,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,hg38_1,185030105,185144793,"[0.015330285954405554, 0.00879942998290062, 0...."
2,hg38_1,36509554,36624242,"[0.0, 0.008249483944382519, 0.0107243291276972..."
3,hg38_1,165722550,165837238,"[0.0034372773370705545, 0.0, 0.0, 0.0273607222..."
4,hg38_1,246758040,246872728,"[0.010999302758136764, 0.0, 0.0, 0.0, 0.0, 0.0..."
5,hg38_1,48426077,48540765,"[0.03856625911430456, 0.009280652084271424, 0...."
6,hg38_1,217497970,217612658,"[0.037122593785170466, 0.013749139907304198, 0..."
7,hg38_1,197777256,197891944,"[0.05458395095774904, 0.05582137260353193, 0.0..."
8,hg38_1,215382076,215496764,"[0.008524463657522574, 0.04399717366322875, 0...."
9,hg38_1,189487854,189602542,"[0.011274294723989442, 0.006599577987799421, 0..."


In [9]:
import ast

In [ ]:
len(ast.literal_eval(bed_df.iloc[1,3])) # length is 896!

896

## Cool, now we can get a dataset. We should be sampling enough to get enough coverage...

In [18]:
320*128

40960

let's try to load enformer.

In [2]:
print('using GPU') if torch.cuda.is_available() else print('using CPU')

using GPU


In [38]:
from enformer_pytorch import Enformer, seq_indices_to_one_hot
model = Enformer.from_hparams(
    dim = 1536,
    depth = 11,
    heads = 8,
    output_heads = dict(human = 5313, mouse = 1643),
    target_length = 896,
).cuda()

In [20]:
seq = torch.randint(0, 5, (1, 196_608)).cuda()
one_hot = seq_indices_to_one_hot(seq)
with torch.no_grad():
    output, embeddings = model(one_hot, return_embeddings = True)

In [24]:
output['human'].cpu().detach().numpy()

array([[[0.7998748 , 0.6254667 , 0.64229167, ..., 0.7191658 ,
         0.76618105, 0.6611264 ],
        [0.7363697 , 0.6093761 , 0.6855796 , ..., 0.7883806 ,
         0.7830241 , 0.61660576],
        [0.854007  , 0.5847356 , 0.72256887, ..., 0.7174225 ,
         0.8242939 , 0.7128622 ],
        ...,
        [0.6512266 , 0.68517524, 0.7045935 , ..., 0.7940127 ,
         0.73009884, 0.64410394],
        [0.65968466, 0.73147565, 0.7095448 , ..., 0.72001946,
         0.7941879 , 0.6191823 ],
        [0.6820772 , 0.6101637 , 0.64959073, ..., 0.6267016 ,
         0.7099414 , 0.6567328 ]]], dtype=float32)

In [30]:
len(seq.cpu().detach().numpy()[0])

196608

In [31]:
len(output['human'].cpu().detach().numpy()[0])

896

In [4]:
from enformer_pytorch import Enformer, seq_indices_to_one_hot, GenomeIntervalDataset
from enformer_loader import GenomeDataIntervalDataset
path = 'tests/data/'
all_chroms = pd.read_csv(path + 'all_chrom_names.txt', sep='\t', header=None)


our files

In [5]:
fasta_file = '/lustre/scratch126/gengen/projects/graft/Dataset/reference/hg38_galGal6_full/fasta/GRCh38.GRCg6a.full.renamed.merged.fa'
bed_file = path + 'test1_train_dataset.bed'

In [98]:
ds = GenomeIntervalDataset(
    bed_file = bed_file,                       # bed file - columns 0, 1, 2 must be <chromosome>, <start position>, <end position>
    fasta_file = fasta_file,                        # path to fasta file                      # filter dataframe function
    return_seq_indices = True,                          # return nucleotide indices (ACGTN) or one hot encodings
    context_length = 196_608,
)

seq = ds[2] # (196608,)
pred = model(seq.cuda()) # if head = 'human', (896, 5313)

OutOfMemoryError: CUDA out of memory. Tried to allocate 576.00 MiB. GPU 0 has a total capacity of 31.73 GiB of which 459.19 MiB is free. Including non-PyTorch memory, this process has 31.28 GiB memory in use. Of the allocated memory 30.79 GiB is allocated by PyTorch, and 129.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [56]:
prednp = pred.cpu().detach().numpy()
prednp.shape

(896, 5313)

In [97]:
prednp[:,1436]

array([0.7633821 , 0.78852516, 0.7381154 , 0.7769616 , 0.70043486,
       0.7532986 , 0.788241  , 0.7081002 , 0.81874466, 0.6937693 ,
       0.67318714, 0.8322517 , 0.76112586, 0.78210145, 0.7183264 ,
       0.7139891 , 0.65781593, 0.79023933, 0.7460313 , 0.6592545 ,
       0.66798127, 0.6463968 , 0.75490975, 0.81654584, 0.59866005,
       0.76725906, 0.71866643, 0.7153028 , 0.85849726, 0.7743673 ,
       0.78089   , 0.73537904, 0.7355636 , 0.7544934 , 0.6835425 ,
       0.77686524, 0.8376123 , 0.80208534, 0.75630915, 0.693698  ,
       0.58794093, 0.7600564 , 0.7194239 , 0.68696666, 0.67759   ,
       0.8083098 , 0.6653601 , 0.74387926, 0.68167895, 0.7879913 ,
       0.7986345 , 0.7195951 , 0.7617609 , 0.75024253, 0.72655493,
       0.75230277, 0.6875209 , 0.73683256, 0.6927696 , 0.7079625 ,
       0.75102514, 0.77746034, 0.7436395 , 0.859849  , 0.6949821 ,
       0.8701673 , 0.733235  , 0.6686225 , 0.70567435, 0.7223281 ,
       0.7641489 , 0.74396557, 0.76451135, 0.68999094, 0.72901

In [66]:
# now with Jacob's code - all_chroms is a list!
all_chroms = pd.read_csv(path + 'all_chrom_names.txt', sep='\t', header=None)
allchromlist = list(all_chroms.iloc[:,0])

In [7]:
allchromlist2 = ['hg38_1']

### Look at this indexing error...

In [44]:
#filter_train = lambda df: df
i = 0
for q in GenomeIntervalDataset(bed_file=bed_file, fasta_file=fasta_file):
    print(i)
    i += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


OutOfBoundsError: index 40 is out of bounds for sequence of length 40

In [22]:
len(list(range(40)))

40

In [46]:
ds = GenomeIntervalDataset(bed_file=bed_file, fasta_file=fasta_file)
len(ds.df)

40

In [48]:
ds.df.row(39)

('hg38_1',
 71837549,
 71952237,
 '[0.007149551995098591, 0.01856128961662762, 0.021517364861210808, 0.0011686742946039885, 0.006118353659985587, 0.029423079453408718, 0.002268603042466566, 0.0047434443986276165, 0.02976682037115097, 0.017598899081349373, 0.0028873193805338815, 0.004812198967556469, 0.01161801820853725, 0.036710134911118075, 0.015605270324158482, 0.009693125510239042, 0.031554198532830924, 0.010518076291191392, 0.0032310406968463212, 0.014986562498961575, 0.01505530254507903, 0.011205539398360997, 0.0, 0.0, 0.006805809127399698, 0.00694330022088252, 0.021654854863299988, 0.019592487733461894, 0.0, 0.008043231806368567, 0.033204110790393315, 0.0, 0.0050184249121230096, 0.008936936428654008, 0.013542876273277216, 0.015192769977147691, 0.01230545599537436, 0.0017873881879495457, 0.011961751719354652, 0.0, 0.009899373093503527, 0.0038497515342896804, 0.0, 0.0, 0.004055987257743254, 0.013886603133869357, 0.01849258816218935, 0.010724329127697274, 0.007837009747163393, 0.010

In [61]:
#
# try and modify .bed file to get only ranges
path = 'tests/data/'
bed_df = pd.read_csv(path + 'test1_train_dataset.bed', sep='\t', header=None)
bed_df

,0,1,2,3
0,hg38_1,144671580,144786268,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,hg38_1,185030105,185144793,"[0.015330285954405554, 0.00879942998290062, 0...."
2,hg38_1,36509554,36624242,"[0.0, 0.008249483944382519, 0.0107243291276972..."
3,hg38_1,165722550,165837238,"[0.0034372773370705545, 0.0, 0.0, 0.0273607222..."
4,hg38_1,246758040,246872728,"[0.010999302758136764, 0.0, 0.0, 0.0, 0.0, 0.0..."
5,hg38_1,48426077,48540765,"[0.03856625911430456, 0.009280652084271424, 0...."
6,hg38_1,217497970,217612658,"[0.037122593785170466, 0.013749139907304198, 0..."
7,hg38_1,197777256,197891944,"[0.05458395095774904, 0.05582137260353193, 0.0..."
8,hg38_1,215382076,215496764,"[0.008524463657522574, 0.04399717366322875, 0...."
9,hg38_1,189487854,189602542,"[0.011274294723989442, 0.006599577987799421, 0..."


In [63]:
for q in bed_df.row:
    print(q)

AttributeError: 'DataFrame' object has no attribute 'row'

In [67]:
for i in range(len(bed_tmp)):
    print(bed_tmp.row(i))
    

('hg38_1', 144671580, 144786268, '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [54]:
test_loader = GenomeDataIntervalDataset(allchromlist2, extend_seq=40960, bed_file = bed_file, fasta_file = fasta_file)

In [51]:
# try to generate a loop for prediction
i = 0
#for sequence, target, loc in test_loader:
for i in range(len(ds.df)):
    print(i)
    # print(f"Sequence is {sequence}") # our test chromosome
    # print(f"target is {target}") # our test chromosome
    # print(f"loc is {loc}") # our test chromosome


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [56]:
test_loader = GenomeIntervalDataset(bed_file = bed_file, fasta_file = fasta_file)

In [60]:
for q in test_loader:
    print(q)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]])
tensor([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.]])
tensor([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
tensor([[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]])
tensor([[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.]])
tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
       

OutOfBoundsError: index 40 is out of bounds for sequence of length 40

custom loader

In [71]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset

import polars as pl
import numpy as np
from random import randrange, random
from pathlib import Path
from pyfaidx import Fasta


def exists(val):
    return val is not None

def identity(t):
    return t

def cast_list(t):
    return t if isinstance(t, list) else [t]

def coin_flip():
    return random() > 0.5

# genomic function transforms

seq_indices_embed = torch.zeros(256).long()
seq_indices_embed[ord('a')] = 0
seq_indices_embed[ord('c')] = 1
seq_indices_embed[ord('g')] = 2
seq_indices_embed[ord('t')] = 3
seq_indices_embed[ord('n')] = 4
seq_indices_embed[ord('A')] = 0
seq_indices_embed[ord('C')] = 1
seq_indices_embed[ord('G')] = 2
seq_indices_embed[ord('T')] = 3
seq_indices_embed[ord('N')] = 4
seq_indices_embed[ord('.')] = -1

one_hot_embed = torch.zeros(256, 4)
one_hot_embed[ord('a')] = torch.Tensor([1., 0., 0., 0.])
one_hot_embed[ord('c')] = torch.Tensor([0., 1., 0., 0.])
one_hot_embed[ord('g')] = torch.Tensor([0., 0., 1., 0.])
one_hot_embed[ord('t')] = torch.Tensor([0., 0., 0., 1.])
one_hot_embed[ord('n')] = torch.Tensor([0., 0., 0., 0.])
one_hot_embed[ord('A')] = torch.Tensor([1., 0., 0., 0.])
one_hot_embed[ord('C')] = torch.Tensor([0., 1., 0., 0.])
one_hot_embed[ord('G')] = torch.Tensor([0., 0., 1., 0.])
one_hot_embed[ord('T')] = torch.Tensor([0., 0., 0., 1.])
one_hot_embed[ord('N')] = torch.Tensor([0., 0., 0., 0.])
one_hot_embed[ord('.')] = torch.Tensor([0.25, 0.25, 0.25, 0.25])

reverse_complement_map = torch.Tensor([3, 2, 1, 0, 4]).long()

def torch_fromstring(seq_strs):
    batched = not isinstance(seq_strs, str)
    seq_strs = cast_list(seq_strs)
    np_seq_chrs = list(map(lambda t: np.fromstring(t, dtype = np.uint8), seq_strs))
    seq_chrs = list(map(torch.from_numpy, np_seq_chrs))
    return torch.stack(seq_chrs) if batched else seq_chrs[0]

def str_to_seq_indices(seq_strs):
    seq_chrs = torch_fromstring(seq_strs)
    return seq_indices_embed[seq_chrs.long()]

def str_to_one_hot(seq_strs):
    seq_chrs = torch_fromstring(seq_strs)
    return one_hot_embed[seq_chrs.long()]

def seq_indices_to_one_hot(t, padding = -1):
    is_padding = t == padding
    t = t.clamp(min = 0)
    one_hot = F.one_hot(t, num_classes = 5)
    out = one_hot[..., :4].float()
    out = out.masked_fill(is_padding[..., None], 0.25)
    return out

# augmentations

def seq_indices_reverse_complement(seq_indices):
    complement = reverse_complement_map[seq_indices.long()]
    return torch.flip(complement, dims = (-1,))

def one_hot_reverse_complement(one_hot):
    *_, n, d = one_hot.shape
    assert d == 4, 'must be one hot encoding with last dimension equal to 4'
    return torch.flip(one_hot, (-1, -2))

class FastaInterval():
    def __init__(
        self,
        *,
        fasta_file,
        context_length = None,
        return_seq_indices = False,
        shift_augs = None,
        rc_aug = False
    ):
        fasta_file = Path(fasta_file)
        assert fasta_file.exists(), 'path to fasta file must exist'

        self.seqs = Fasta(str(fasta_file))
        self.return_seq_indices = return_seq_indices
        self.context_length = context_length
        self.shift_augs = shift_augs
        self.rc_aug = rc_aug

    def __call__(self, chr_name, start, end, return_augs = False):
        interval_length = end - start
        chromosome = self.seqs[chr_name]
        chromosome_length = len(chromosome)

        if exists(self.shift_augs):
            min_shift, max_shift = self.shift_augs
            max_shift += 1

            min_shift = min(max(start + min_shift, 0) - start, 0)
            max_shift = max(min(end + max_shift, chromosome_length) - end, 1)

            rand_shift = randrange(min_shift, max_shift)
            start += rand_shift
            end += rand_shift

        left_padding = right_padding = 0

        if exists(self.context_length) and interval_length < self.context_length:
            extra_seq = self.context_length - interval_length

            extra_left_seq = extra_seq // 2
            extra_right_seq = extra_seq - extra_left_seq

            start -= extra_left_seq
            end += extra_right_seq

        if start < 0:
            left_padding = -start
            start = 0

        if end > chromosome_length:
            right_padding = end - chromosome_length
            end = chromosome_length

        seq = ('.' * left_padding) + str(chromosome[start:end]) + ('.' * right_padding)

        should_rc_aug = self.rc_aug and coin_flip()

        if self.return_seq_indices:
            seq = str_to_seq_indices(seq)

            if should_rc_aug:
                seq = seq_indices_reverse_complement(seq)

            return seq

        one_hot = str_to_one_hot(seq)

        if should_rc_aug:
            one_hot = one_hot_reverse_complement(one_hot)

        if not return_augs:
            return one_hot

        # returns the shift integer as well as the bool (for whether reverse complement was activated)
        # for this particular genomic sequence

        rand_shift_tensor = torch.tensor([rand_shift])
        rand_aug_bool_tensor = torch.tensor([should_rc_aug])

        return one_hot, rand_shift_tensor, rand_aug_bool_tensor

def identity(t):
    return t

In [109]:
class GenomeIntervalDataset(Dataset):
    def __init__(
        self,
        bed_file,
        fasta_file,
        filter_df_fn = identity,
        chr_bed_to_fasta_map = dict(),
        context_length = None,
        return_seq_indices = False,
        shift_augs = None,
        rc_aug = False,
        return_augs = False
    ):
        super().__init__()
        bed_path = Path(bed_file)
        assert bed_path.exists(), 'path to .bed file must exist'

        df = pd.read_csv(str(bed_path), sep = '\t', header = None)
        df = filter_df_fn(df)
        self.df = df

        # if the chromosome name in the bed file is different than the keyname in the fasta
        # can remap on the fly
        self.chr_bed_to_fasta_map = chr_bed_to_fasta_map

        self.fasta = FastaInterval(
            fasta_file = fasta_file,
            context_length = context_length,
            return_seq_indices = return_seq_indices,
            shift_augs = shift_augs,
            rc_aug = rc_aug
        )

        self.return_augs = return_augs

    def __len__(self):
        return len(self.df)

    def __getitem__(self, ind):
        interval = self.df.iloc[ind, :]
        chr_name, start, end = (interval.iloc[0], interval.iloc[1], interval.iloc[2])
        chr_name = self.chr_bed_to_fasta_map.get(chr_name, chr_name)
        return self.fasta(chr_name, start, end, return_augs = self.return_augs)


In [115]:
import polars as pl
bed_tmp = pl.read_csv(str(bed_file), separator = '\t', has_header = False)
bed_tmp

column_1,column_2,column_3,column_4
str,i64,i64,str
"""hg38_1""",144671580,144786268,"""[0.0, 0.0, 0.0, 0.0, 0.0, 0.0,…"
"""hg38_1""",185030105,185144793,"""[0.015330285954405554, 0.00879…"
"""hg38_1""",36509554,36624242,"""[0.0, 0.008249483944382519, 0.…"
"""hg38_1""",165722550,165837238,"""[0.0034372773370705545, 0.0, 0…"
"""hg38_1""",246758040,246872728,"""[0.010999302758136764, 0.0, 0.…"
…,…,…,…
"""hg38_1""",162939824,163054512,"""[0.048259384668199345, 0.01649…"
"""hg38_1""",151859826,151974514,"""[0.04138483476708643, 0.015399…"
"""hg38_1""",1620614,1735302,"""[0.06159601075341925, 0.012924…"


In [97]:
bed_tmp.row(39)

('hg38_1',
 71837549,
 71952237,
 '[0.007149551995098591, 0.01856128961662762, 0.021517364861210808, 0.0011686742946039885, 0.006118353659985587, 0.029423079453408718, 0.002268603042466566, 0.0047434443986276165, 0.02976682037115097, 0.017598899081349373, 0.0028873193805338815, 0.004812198967556469, 0.01161801820853725, 0.036710134911118075, 0.015605270324158482, 0.009693125510239042, 0.031554198532830924, 0.010518076291191392, 0.0032310406968463212, 0.014986562498961575, 0.01505530254507903, 0.011205539398360997, 0.0, 0.0, 0.006805809127399698, 0.00694330022088252, 0.021654854863299988, 0.019592487733461894, 0.0, 0.008043231806368567, 0.033204110790393315, 0.0, 0.0050184249121230096, 0.008936936428654008, 0.013542876273277216, 0.015192769977147691, 0.01230545599537436, 0.0017873881879495457, 0.011961751719354652, 0.0, 0.009899373093503527, 0.0038497515342896804, 0.0, 0.0, 0.004055987257743254, 0.013886603133869357, 0.01849258816218935, 0.010724329127697274, 0.007837009747163393, 0.010

In [110]:
bed_df = pd.read_csv(str(bed_file), sep = '\t', header = None)
bed_df


,0,1,2,3
0,hg38_1,144671580,144786268,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,hg38_1,185030105,185144793,"[0.015330285954405554, 0.00879942998290062, 0...."
2,hg38_1,36509554,36624242,"[0.0, 0.008249483944382519, 0.0107243291276972..."
3,hg38_1,165722550,165837238,"[0.0034372773370705545, 0.0, 0.0, 0.0273607222..."
4,hg38_1,246758040,246872728,"[0.010999302758136764, 0.0, 0.0, 0.0, 0.0, 0.0..."
5,hg38_1,48426077,48540765,"[0.03856625911430456, 0.009280652084271424, 0...."
6,hg38_1,217497970,217612658,"[0.037122593785170466, 0.013749139907304198, 0..."
7,hg38_1,197777256,197891944,"[0.05458395095774904, 0.05582137260353193, 0.0..."
8,hg38_1,215382076,215496764,"[0.008524463657522574, 0.04399717366322875, 0...."
9,hg38_1,189487854,189602542,"[0.011274294723989442, 0.006599577987799421, 0..."


In [132]:
bed_df.iloc[39, :]

0                                               hg38_1
1                                             71837549
2                                             71952237
3    [0.007149551995098591, 0.01856128961662762, 0....
Name: 39, dtype: object

In [111]:
test_loader = GenomeIntervalDataset(bed_file = bed_file, fasta_file = fasta_file)

In [114]:
i = 0
for q in test_loader:
    i += 1
    print(i)
    print(q)


/tmp/ipykernel_3144979/1631023882.py:57: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np_seq_chrs = list(map(lambda t: np.fromstring(t, dtype = np.uint8), seq_strs))


1
tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]])
2
tensor([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.]])
3
tensor([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
4
tensor([[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]])
5
tensor([[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.]])
6
tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 0., 0., 

In [121]:
# original
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset

import polars as pl
import numpy as np
from random import randrange, random
from pathlib import Path
from pyfaidx import Fasta

# helper functions

def exists(val):
    return val is not None

def identity(t):
    return t

def cast_list(t):
    return t if isinstance(t, list) else [t]

def coin_flip():
    return random() > 0.5

# genomic function transforms

seq_indices_embed = torch.zeros(256).long()
seq_indices_embed[ord('a')] = 0
seq_indices_embed[ord('c')] = 1
seq_indices_embed[ord('g')] = 2
seq_indices_embed[ord('t')] = 3
seq_indices_embed[ord('n')] = 4
seq_indices_embed[ord('A')] = 0
seq_indices_embed[ord('C')] = 1
seq_indices_embed[ord('G')] = 2
seq_indices_embed[ord('T')] = 3
seq_indices_embed[ord('N')] = 4
seq_indices_embed[ord('.')] = -1

one_hot_embed = torch.zeros(256, 4)
one_hot_embed[ord('a')] = torch.Tensor([1., 0., 0., 0.])
one_hot_embed[ord('c')] = torch.Tensor([0., 1., 0., 0.])
one_hot_embed[ord('g')] = torch.Tensor([0., 0., 1., 0.])
one_hot_embed[ord('t')] = torch.Tensor([0., 0., 0., 1.])
one_hot_embed[ord('n')] = torch.Tensor([0., 0., 0., 0.])
one_hot_embed[ord('A')] = torch.Tensor([1., 0., 0., 0.])
one_hot_embed[ord('C')] = torch.Tensor([0., 1., 0., 0.])
one_hot_embed[ord('G')] = torch.Tensor([0., 0., 1., 0.])
one_hot_embed[ord('T')] = torch.Tensor([0., 0., 0., 1.])
one_hot_embed[ord('N')] = torch.Tensor([0., 0., 0., 0.])
one_hot_embed[ord('.')] = torch.Tensor([0.25, 0.25, 0.25, 0.25])

reverse_complement_map = torch.Tensor([3, 2, 1, 0, 4]).long()

def torch_fromstring(seq_strs):
    batched = not isinstance(seq_strs, str)
    seq_strs = cast_list(seq_strs)
    np_seq_chrs = list(map(lambda t: np.fromstring(t, dtype = np.uint8), seq_strs))
    seq_chrs = list(map(torch.from_numpy, np_seq_chrs))
    return torch.stack(seq_chrs) if batched else seq_chrs[0]

def str_to_seq_indices(seq_strs):
    seq_chrs = torch_fromstring(seq_strs)
    return seq_indices_embed[seq_chrs.long()]

def str_to_one_hot(seq_strs):
    seq_chrs = torch_fromstring(seq_strs)
    return one_hot_embed[seq_chrs.long()]

def seq_indices_to_one_hot(t, padding = -1):
    is_padding = t == padding
    t = t.clamp(min = 0)
    one_hot = F.one_hot(t, num_classes = 5)
    out = one_hot[..., :4].float()
    out = out.masked_fill(is_padding[..., None], 0.25)
    return out

# augmentations

def seq_indices_reverse_complement(seq_indices):
    complement = reverse_complement_map[seq_indices.long()]
    return torch.flip(complement, dims = (-1,))

def one_hot_reverse_complement(one_hot):
    *_, n, d = one_hot.shape
    assert d == 4, 'must be one hot encoding with last dimension equal to 4'
    return torch.flip(one_hot, (-1, -2))

# processing bed files

class FastaInterval():
    def __init__(
        self,
        *,
        fasta_file,
        context_length = None,
        return_seq_indices = False,
        shift_augs = None,
        rc_aug = False
    ):
        fasta_file = Path(fasta_file)
        assert fasta_file.exists(), 'path to fasta file must exist'

        self.seqs = Fasta(str(fasta_file))
        self.return_seq_indices = return_seq_indices
        self.context_length = context_length
        self.shift_augs = shift_augs
        self.rc_aug = rc_aug

    def __call__(self, chr_name, start, end, return_augs = False):
        interval_length = end - start
        chromosome = self.seqs[chr_name]
        chromosome_length = len(chromosome)

        if exists(self.shift_augs):
            min_shift, max_shift = self.shift_augs
            max_shift += 1

            min_shift = min(max(start + min_shift, 0) - start, 0)
            max_shift = max(min(end + max_shift, chromosome_length) - end, 1)

            rand_shift = randrange(min_shift, max_shift)
            start += rand_shift
            end += rand_shift

        left_padding = right_padding = 0

        if exists(self.context_length) and interval_length < self.context_length:
            extra_seq = self.context_length - interval_length

            extra_left_seq = extra_seq // 2
            extra_right_seq = extra_seq - extra_left_seq

            start -= extra_left_seq
            end += extra_right_seq

        if start < 0:
            left_padding = -start
            start = 0

        if end > chromosome_length:
            right_padding = end - chromosome_length
            end = chromosome_length

        seq = ('.' * left_padding) + str(chromosome[start:end]) + ('.' * right_padding)

        should_rc_aug = self.rc_aug and coin_flip()

        if self.return_seq_indices:
            seq = str_to_seq_indices(seq)

            if should_rc_aug:
                seq = seq_indices_reverse_complement(seq)

            return seq

        one_hot = str_to_one_hot(seq)

        if should_rc_aug:
            one_hot = one_hot_reverse_complement(one_hot)

        if not return_augs:
            return one_hot

        # returns the shift integer as well as the bool (for whether reverse complement was activated)
        # for this particular genomic sequence

        rand_shift_tensor = torch.tensor([rand_shift])
        rand_aug_bool_tensor = torch.tensor([should_rc_aug])

        return one_hot, rand_shift_tensor, rand_aug_bool_tensor


class GenomeIntervalDataset(Dataset):
    def __init__(
        self,
        bed_file,
        fasta_file,
        filter_df_fn = identity,
        chr_bed_to_fasta_map = dict(),
        context_length = None,
        return_seq_indices = False,
        shift_augs = None,
        rc_aug = False,
        return_augs = False
    ):
        super().__init__()
        bed_path = Path(bed_file)
        assert bed_path.exists(), 'path to .bed file must exist'

        df = pl.read_csv(str(bed_path), separator = '\t', has_header = False)
        df = filter_df_fn(df)
        self.df = df

        # if the chromosome name in the bed file is different than the keyname in the fasta
        # can remap on the fly
        self.chr_bed_to_fasta_map = chr_bed_to_fasta_map

        self.fasta = FastaInterval(
            fasta_file = fasta_file,
            context_length = context_length,
            return_seq_indices = return_seq_indices,
            shift_augs = shift_augs,
            rc_aug = rc_aug
        )

        self.return_augs = return_augs

    def __len__(self):
        return len(self.df)

    def __getitem__(self, ind):
        interval = self.df.row(ind)
        chr_name, start, end = (interval[0], interval[1], interval[2])
        chr_name = self.chr_bed_to_fasta_map.get(chr_name, chr_name)
        return self.fasta(chr_name, start, end, return_augs = self.return_augs)


In [127]:
# what about the train filter???
import polars as pl
bed_tmp = pl.read_csv(str(bed_file), separator = '\t', has_header = False)

filter_train = identity
bed_tmp = filter_train(bed_tmp)
bed_tmp

column_1,column_2,column_3,column_4
str,i64,i64,str
"""hg38_1""",144671580,144786268,"""[0.0, 0.0, 0.0, 0.0, 0.0, 0.0,…"
"""hg38_1""",185030105,185144793,"""[0.015330285954405554, 0.00879…"
"""hg38_1""",36509554,36624242,"""[0.0, 0.008249483944382519, 0.…"
"""hg38_1""",165722550,165837238,"""[0.0034372773370705545, 0.0, 0…"
"""hg38_1""",246758040,246872728,"""[0.010999302758136764, 0.0, 0.…"
…,…,…,…
"""hg38_1""",162939824,163054512,"""[0.048259384668199345, 0.01649…"
"""hg38_1""",151859826,151974514,"""[0.04138483476708643, 0.015399…"
"""hg38_1""",1620614,1735302,"""[0.06159601075341925, 0.012924…"


In [129]:
bed_tmp.row(40)

OutOfBoundsError: index 40 is out of bounds for sequence of length 40

In [ ]:
test_loader = GenomeIntervalDataset(bed_file = bed_file, fasta_file = fasta_file, 
                                    filter_df_fn = filter_train)
i = 0
for q in test_loader:
    i += 1
    print(i)
    print(q)

### Let's simply predict for all our .bed file

In [133]:
allchromlist2 = ['hg38_1']

In [144]:
from enformer_pytorch import Enformer, seq_indices_to_one_hot, GenomeIntervalDataset
from enformer_loader import GenomeDataIntervalDataset
import ast
test_loader = GenomeDataIntervalDataset(allchromlist2, extend_seq=40960, bed_file = bed_file, fasta_file = fasta_file)

In [145]:
# modified to use pandas
class GenomeIntervalDataset(Dataset):
    def __init__(
        self,
        bed_file,
        fasta_file,
        filter_df_fn = identity,
        chr_bed_to_fasta_map = dict(),
        context_length = None,
        return_seq_indices = False,
        shift_augs = None,
        rc_aug = False,
        return_augs = False
    ):
        super().__init__()
        bed_path = Path(bed_file)
        assert bed_path.exists(), 'path to .bed file must exist'

        df = pd.read_csv(str(bed_path), sep = '\t', header = None)
        df = filter_df_fn(df)
        self.df = df

        # if the chromosome name in the bed file is different than the keyname in the fasta
        # can remap on the fly
        self.chr_bed_to_fasta_map = chr_bed_to_fasta_map

        self.fasta = FastaInterval(
            fasta_file = fasta_file,
            context_length = context_length,
            return_seq_indices = return_seq_indices,
            shift_augs = shift_augs,
            rc_aug = rc_aug
        )

        self.return_augs = return_augs

    def __len__(self):
        return len(self.df)

    def __getitem__(self, ind):
        interval = self.df.iloc[ind, :]
        chr_name, start, end = (interval.iloc[0], interval.iloc[1], interval.iloc[2])
        chr_name = self.chr_bed_to_fasta_map.get(chr_name, chr_name)
        return self.fasta(chr_name, start, end, return_augs = self.return_augs)


class GenomeDataIntervalDataset(GenomeIntervalDataset):
    def __init__(self, all_chroms, extend_seq=0, *args, **kwargs):
        """
        Argument all_chroms is a list of chromosomes in the dataset.
        E.g. ['chr1', 'chr2', ...].

        Argument extend_seq specifies how many basepairs to extend the sequence
        on either side by. Default = 0. Note that you might need to specify
        the data generation process with generate_dataset.py as well so that
        the sampled regions always have enough padding along the sides of the
        region to allow for the extend_seq amount. Use argument --padding in
        those scripts for this.
        """
        super(GenomeDataIntervalDataset, self).__init__(*args, **kwargs)
        self.chr_to_int = dict(zip(all_chroms, range(len(all_chroms))))
        self.int_to_chr = {v: k for k, v in self.chr_to_int.items()}
        self.extend_seq = extend_seq

    def __len__(self):
        return len(self.df)

    def __getitem__(self, ind):
        interval = self.df.iloc[ind,:]
        chr_name, start, end, target = (
            interval[0], interval[1], interval[2], interval[3])
        chr_name = self.chr_bed_to_fasta_map.get(chr_name, chr_name)
        target = ast.literal_eval(target)
        target = torch.tensor(target).unsqueeze(-1)

        # This will break if you do not leave enough space for the padding:
        ext_start = start - self.extend_seq
        ext_end = end + self.extend_seq

        sequence = self.fasta(chr_name, ext_start, ext_end,
                              return_augs=self.return_augs)
        loc = torch.tensor([self.chr_to_int[chr_name], start, end])
        return sequence, target, loc

    @staticmethod
    def check_tensor_dtype(tensor):
        if tensor.dtype == torch.float16:
            return "fp16"
        elif tensor.dtype == torch.bfloat16:
            return "bf16"
        else:
            return str(tensor.dtype)

test_loader = GenomeDataIntervalDataset(allchromlist2, extend_seq=40960, bed_file = bed_file, fasta_file = fasta_file)

we can specify the model like this...

In [ ]:
from enformer_pytorch import Enformer, seq_indices_to_one_hot
print('using GPU') if torch.cuda.is_available() else print('using CPU')
model = Enformer.from_hparams(
    dim = 1536,
    depth = 11,
    heads = 8,
    output_heads = dict(human = 5313, mouse = 1643),
    target_length = 896,
).cuda()

or like this for the pretrained model

In [ ]:
weight_file = '/lustre/scratch126/gengen/teams/parts/jh47/enformer_pytorch/model_weights/'
from enformer_pytorch import from_pretrained
from enformer_pytorch.finetune import HeadAdapterWrapper

enformer = from_pretrained(weight_file, use_checkpointing = True).cuda()

In [ ]:
seq = torch.randint(0, 5, (1, 196_608)).cuda()
one_hot = seq_indices_to_one_hot(seq)
with torch.no_grad():
    output, embeddings = enformer(one_hot, return_embeddings = True)

/nfs/users/nfs_s/sb79/.local/lib/python3.12/site-packages/torch/utils/checkpoint.py:553: UserWarning: torch.utils.checkpoint.checkpoint_sequential: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/nfs/users/nfs_s/sb79/.local/lib/python3.12/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
# predict random
seq = torch.randint(0, 5, (1, 196_608)).cuda()
one_hot = seq_indices_to_one_hot(seq)
with torch.no_grad():
    output, embeddings = model(one_hot, return_embeddings = True)

In [ ]:
o = output['human'].cpu().detach().squeeze(0).numpy()
len(o[:, 1436]) # the HEK track we are interested in

896

let's run all our .bed file

In [200]:
type(test_loader[0][0].long())

/tmp/ipykernel_3144979/2220128533.py:59: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np_seq_chrs = list(map(lambda t: np.fromstring(t, dtype = np.uint8), seq_strs))


torch.Tensor

In [203]:
test_loader[0].long()

/tmp/ipykernel_3144979/2220128533.py:59: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np_seq_chrs = list(map(lambda t: np.fromstring(t, dtype = np.uint8), seq_strs))


AttributeError: 'tuple' object has no attribute 'long'

In [189]:
seq = torch.randint(0, 5, (1, 196_608))
type(seq)

torch.Tensor

In [199]:
seq.cpu().detach().numpy()[0][0:10]

array([1, 4, 4, 0, 0, 3, 4, 4, 1, 2])

In [204]:
with torch.no_grad():
    y_hat_vector = []
    for seq, target, loc in test_loader:
        s = seq.cuda()
        y_hat = enformer(s)
        y_hat_vector.append(y_hat)



/tmp/ipykernel_3144979/2220128533.py:59: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np_seq_chrs = list(map(lambda t: np.fromstring(t, dtype = np.uint8), seq_strs))
/nfs/users/nfs_s/sb79/.local/lib/python3.12/site-packages/torch/utils/checkpoint.py:553: UserWarning: torch.utils.checkpoint.checkpoint_sequential: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/nfs/users/nfs_s/sb79/.local/lib/python3.12/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
predtest = []
for i in range(len(y_hat_vector)):
    predtest.append(y_hat_vector[i]['human'][:,1436].cpu().detach().squeeze(0).numpy())

In [231]:
len(predtest[39])

896

## Now we can extract our desired tracks from the prediction. Let's generate a real dataset for chromosome 1 and predict it.

In [234]:
248956422 / 196608

1266.2578430175781

### Try and finetune with imported weights

In [93]:
model = HeadAdapterWrapper(
    enformer = enformer,
    num_tracks = 1,
    post_transformer_embed = False   # by default, embeddings are taken from after the final pointwise block w/ conv -> gelu - but if you'd like the embeddings right after the transformer block with a learned layernorm, set this to True
).cuda()

model # we can see the extra layer added at the end

HeadAdapterWrapper(
  (enformer): Enformer(
    (stem): Sequential(
      (0): Conv1d(4, 768, kernel_size=(15,), stride=(1,), padding=(7,))
      (1): Residual(
        (fn): Sequential(
          (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): GELU()
          (2): Conv1d(768, 768, kernel_size=(1,), stride=(1,))
        )
      )
      (2): AttentionPool(
        (pool_fn): Rearrange('b d (n p) -> b d n p', p=2)
        (to_attn_logits): Conv2d(768, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
    (conv_tower): Sequential(
      (0): Sequential(
        (0): Sequential(
          (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): GELU()
          (2): Conv1d(768, 768, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): Residual(
          (fn): Sequential(
            (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=